In [1]:
from collections import defaultdict
import os
from pathlib import Path
from pprint import pprint
import re
import shutil
from uuid import uuid4
from itertools import chain

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from schematics.types import IntType, StringType, ListType

%cd /root/hidebound/python
import hidebound.tools as tools
from hidebound.parser import AssetNameParser
from hidebound.database import Database
import hidebound.database_tools as db_tools
from hidebound.database_test import DatabaseTests
from hidebound.specification_base import *
from hidebound.specifications import *
from hidebound.validators import *
from hidebound.traits import *

/root/hidebound/python


In [4]:
root = '/tmp/projects'
if Path(root).exists():
    shutil.rmtree(root)
os.makedirs(root)
fullpaths = DatabaseTests().create_files(root)

hb_root = '/tmp/hidebound'
if Path(hb_root).exists():
    shutil.rmtree(hb_root)
os.makedirs(hb_root)

In [5]:
!tree /tmp/hidebound/; tree /tmp/projects

/tmp/hidebound/

0 directories, 0 files
/tmp/projects
├── proj001
│   ├── spec001
│   │   └── pizza
│   │       ├── p-proj001_s-spec001_d-pizza_v001
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│   │       ├── p-proj001_s-spec001_d-pizza_v002
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0003.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0004.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0003.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v002_c000-001_f0004.png
│   │     

In [6]:
class Spec001(SequenceSpecificationBase):
    name = 'spec001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']
    descriptor = ListType(StringType(), required=True, validators=[is_descriptor, is_homogenous])
    frame = ListType(IntType(), required=True, validators=[is_frame])
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'png')])
    coordinate = ListType(ListType(IntType(), required=True, validators=[is_coordinate]))
#     file_traits = dict(
#         width=get_image_width
#     )

class Spec002(SequenceSpecificationBase):
    name = 'spec002'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'frame', 'extension']
    width = ListType(IntType(), required=True, validators=[lambda x: is_eq(x, 1024)])
    height = ListType(IntType(), required=True, validators=[lambda x: is_eq(x, 1024)])
    frame = ListType(IntType(), required=True, validators=[is_frame, lambda x: is_gt(x, -1)])
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'exr')])
    file_traits = dict(
        width=get_image_width
    )
    
class Vdb001(FileSpecificationBase):
    name = 'vdb001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'extension']
    extension = StringType(required=True, validators=[is_extension, lambda x: is_eq(x, 'vdb')])

# db = Database(root, [Spec001, Spec002, Vdb001])
# db = Database(root, [Spec001, Spec002], exclude_regex='misc\.txt|vdb001')
root = '/tmp/projects'
hb_root = '/tmp'
os.makedirs(hb_root, exist_ok=True)
db = Database(root, hb_root, [Spec001, Spec002])
db.update()
x = db.create()
x
# x.asset_id.apply(lambda x: x)
# db.data.head(10)

!tree /tmp/hidebound/; tree /tmp/projects

/tmp/hidebound/
├── data
│   └── proj001
│       └── spec001
│           └── pizza
│               ├── p-proj001_s-spec001_d-pizza_v001
│               │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│               │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│               │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│               └── p-proj001_s-spec001_d-pizza_v002
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0001.png
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0002.png
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0003.png
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0004.png
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0001.png
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0002.png
│                   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0003.png
│                   └── p-proj001_s-spec001_d-pizza

In [22]:
db.data.query('version == 3')

,project,specification,descriptor,version,coordinate,frame,extension,filename,filepath,file_error,file_traits,asset_name,asset_path,asset_type,asset_traits,asset_error,asset_valid,width,width_error
14,proj001,spec001,kiwi,3.0,"[0, 1]",1.0,png,p-proj001_s-spec001_d-kiwi_v003_c000-001_f0001...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,ValidationError( ['Invalid asset directory nam...,"{'project': 'proj001', 'specification': 'spec0...",NaN,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence,{'filename_error': ['ParseException( Illegal d...,DataError(\n{'filename_error': 'Rogue field'}\n),False,NaN,NaN
15,proj001,spec001,pizza,3.0,"[0, 1]",2.0,png,p-proj001_s-spec001_d-pizza_v003_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,"{'project': 'proj001', 'specification': 'spec0...",p-proj001_s-spec001_d-pizza_v003,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence,{'filename_error': ['ParseException( Illegal d...,DataError(\n{'filename_error': 'Rogue field'}\n),False,NaN,NaN
16,proj001,spec001,pizza,3.0,"[0, 1]",4.0,png,p-proj001_s-spec001_d-pizza_v003_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,"{'project': 'proj001', 'specification': 'spec0...",p-proj001_s-spec001_d-pizza_v003,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence,{'filename_error': ['ParseException( Illegal d...,DataError(\n{'filename_error': 'Rogue field'}\n),False,NaN,NaN


In [63]:
import pandasql
data = db.data.drop(columns=['asset_traits', 'file_traits', 'coordinate'])
pandasql.sqldf('SELECT * FROM data WHERE width_error is not NULL;', {'data': data})

,project,specification,descriptor,version,frame,extension,filename,filepath,file_error,asset_name,asset_path,asset_type,asset_error,asset_valid,width,width_error
0,None,spec002,None,None,None,,p-proj001_s-spec002_d-taco_v002,/tmp/projects/proj001/spec002/taco/p-proj001_s...,ParseException( p-proj001_s-spec002_d-taco_v002 ),None,/tmp/projects/proj001/spec002/taco/p-proj001_s...,sequence,"DataError(\n{'filename_error': 'Rogue field', ...",0,None,ValueError( Could not find a format to read th...


In [10]:
data = db.data
asset_errors = data[data.file_error.notnull()].copy()
asset_errors['file_errors'] = asset_errors.apply(lambda x: [x.filepath, x.file_error], axis=1)
asset_errors = asset_errors.groupby('asset_path', as_index=False).file_errors.agg(lambda x: x.tolist())
length = 500
print('-' * length)
for i, row in asset_errors.iterrows():
    print(row.asset_path)
    for item in row.file_errors:
        print(f'\t{item[0]:<150}{item[1]}')
    print('-' * length)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003
	/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003/misc.txt                                                                         Specification not found in "misc.txt". (at char 0), (line:1, col:1)
	/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003/p-proj001_s-spec0001_d-pizza_v003_c000-001_f0005.png                             KeyError( Specification not found. )
	/tmp/projects/proj001/

In [5]:
def _groupby_asset_path(self):
    cols = ['asset_path', 'traits', 'specification']
    data = self.data[cols]\
        .groupby('asset_path', as_index=False)\
        .agg(lambda x: x.tolist())import json
x = '/tmp/assets/metadata/asset'
x = [Path(x, y) for y in os.listdir(x)][0]
with open(x) as f:
    x = json.load(f)
x
    data.traits = data.traits.apply(to_prototype)
    data.specification = data.specification.apply(lambda x: list(set(x)))

    data['asset_error'] = np.nan
    mask = data.specification.apply(lambda x: len(x) > 1)
    data.loc[mask, 'asset_error'] = 'Multiple specifications found.'
    
    mask = data.specification.apply(lambda x: len(x) == 0)
    data.loc[mask, 'asset_error'] = 'Specification not found.'
    
    mask = data.specification.apply(lambda x: len(x) == 1 and x[0] not in self._specifications.keys())
    data.loc[mask, 'asset_error'] = 'Specification not found.'
    
    mask = data.specification.apply(lambda x: len(x) == 1)
    data.loc[mask, 'specification'] = data.loc[mask, 'specification'].apply(lambda x: x[0])

    return data